# AWS Bedrock AgentCore Gateway - Lambda to MCP Tools

This notebook demonstrates how to create an AWS Bedrock AgentCore Gateway that exposes AWS Lambda functions as Model Context Protocol (MCP) tools, enabling seamless integration with AI agents using Strands Agents.

## Overview

This tutorial covers:
- Creating an AgentCore Gateway with Cognito-based OAuth authentication (EZAuth flow)
- Registering Lambda functions as MCP tools
- Connecting Strands Agents to the gateway
- Invoking Lambda-backed tools through AI agents

## Prerequisites

Before running this notebook, ensure you have:

1. **Python 3.12+** installed
2. **AWS CLI** configured with appropriate credentials
3. **AWS Account** with permissions for:
   - Bedrock AgentCore Gateway
   - AWS Lambda
   - Amazon Cognito
   - IAM (for role creation)
4. **AWS Profile** configured (or use default credentials)

---


In [ ]:
!pip install bedrock_agentcore_starter_toolkit strands-agents strands-agents-tools

## Step 2: Import Required Libraries

Import the necessary Python libraries for AWS operations, environment management, and logging.

In [ ]:
import boto3
import os
import logging
import json

## Step 3: Configure AWS Environment

Set your AWS region and profile. **Note**: Replace these values with your own AWS configuration.

- `AWS_DEFAULT_REGION`: Your AWS region (e.g., 'us-east-1', 'eu-central-1')
- `AWS_PROFILE`: Your AWS profile name (optional, uses default if not specified)

In [ ]:
# Configure AWS region and profile
# Replace with your AWS region and profile name
os.environ['AWS_DEFAULT_REGION'] = 'eu-central-1'  # Change to your region
os.environ['AWS_PROFILE'] = 'your-aws-profile'  # Optional: Change to your profile or remove to use default

## Step 4: Create AgentCore Gateway with Cognito Authentication

This step creates an AgentCore Gateway using the EZAuth flow, which automatically sets up:
- Amazon Cognito User Pool
- Cognito Domain
- OAuth 2.0 client configuration
- IAM execution role for the gateway

The gateway will expose Lambda functions as MCP tools that can be discovered and invoked by AI agents.

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient

# Initialize the Gateway client
client = GatewayClient(region_name=os.environ['AWS_DEFAULT_REGION'])
client.logger.setLevel(logging.DEBUG)

# Create Cognito Authorizer using EZAuth flow
# This automatically creates Cognito User Pool, domain, and OAuth client
cognito_authorizer = client.create_oauth_authorizer_with_cognito("agentcore-gateway-test")

# Create the MCP Gateway with Cognito authentication
gateway = client.create_mcp_gateway(authorizer_config=cognito_authorizer["authorizer_config"])

# Extract gateway identifiers
gatewayID = gateway["gatewayId"]
gatewayUrl = gateway["gatewayUrl"]

print(f"✓ Gateway created successfully!")
print(f"Gateway ID: {gatewayID}")
print(f"Gateway URL: {gatewayUrl}")
print(f"\nNote: The gateway URL will be used to connect MCP clients.")

## Create Lambda target

In [ ]:
# This helper will create a lambda by default
lambda_target = client.create_mcp_gateway_target(gateway=gateway, target_type="lambda")

## Adding another Lambda 

In [ ]:
lambda_target_configuration = {
    'lambdaArn': 'arn:aws:lambda:eu-central-1:911101829662:function:agentCoreGatewayCustomLambda',
    'toolSchema': {
        # 's3': {
        #     'uri': 'string',
        #     'bucketOwnerAccountId': 'string'
        # },
        'inlinePayload': [
            {
                'name': 'get_random_number',
                'description': 'Returning a random number',
                'inputSchema': {
                    'type': 'object',
                    'properties': {},
                    'required': []
                },
                'outputSchema': {
                    'type': 'integer'
                }
            },
        ]
    }
}

response = client.create_mcp_gateway_target(
    gateway=gateway,
    target_type="lambda",
    target_payload=lambda_target_configuration
)

print("✓ Lambda target added\n")
print(f"Lambda Target = {response}")

## Call the MCP tools from Strands Agents

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp.mcp_client import MCPClient
from mcp.client.streamable_http import streamablehttp_client 

# Refresh the access token to ensure it's valid
access_token = client.get_access_token_for_cognito(cognito_authorizer["client_info"])
print(f"✓ Refreshed access token")

# Ensure the gateway URL ends with /mcp
mcp_url = gatewayUrl if gatewayUrl.endswith('/mcp') else f"{gatewayUrl}/mcp"
print(f"✓ Using gateway URL: {mcp_url}")

streamable_http_mcp_client = MCPClient(
    lambda: streamablehttp_client(
        url=mcp_url,
        headers={"Authorization": f"Bearer {access_token}"}
    )    
)

# Start the MCP client and keep it running
streamable_http_mcp_client.start()
print(f"✓ MCP client started")

# Get tools while client is running
tools = streamable_http_mcp_client.list_tools_sync()
print(f"✓ Successfully connected! Found {len(tools)} tools")

# Create agent with the tools
model = BedrockModel(model_id="eu.amazon.nova-pro-v1:0")
agent = Agent(model=model, tools=tools)
print(f"Tools loaded in the agent are {agent.tool_names}")
print(f"\n⚠️  IMPORTANT: Keep the MCP client running while using the agent!")
print(f"   Use streamable_http_mcp_client.stop() when done.")




## Invoking the agent

In [ ]:
# Make sure the MCP client is still running (it should be from Cell 10)
# The agent needs the client session to be active to call tools
response = agent("Get the time for PST")
print("Response: ", response)

## Clean up

In [ ]:
# Stop the MCP client if it's running
try:
    if streamable_http_mcp_client.is_running():
        streamable_http_mcp_client.stop()
        print("✓ MCP client stopped")
except:
    pass

# Clean up gateway resources
client.delete_gateway_target(gatewayID, lambda_target["targetId"])
client.delete_gateway(gatewayID)
print("✓ Gateway resources cleaned up")

## Summary

This notebook demonstrated how to:

1. ✅ **Create an AgentCore Gateway** with Cognito-based OAuth authentication
2. ✅ **Register Lambda functions** as MCP tools with custom schemas
3. ✅ **Connect Strands Agents** to discover and use Lambda-backed tools
4. ✅ **Invoke tools** through AI agents using natural language

## Key Takeaways

- **AgentCore Gateway** bridges AWS Lambda functions and AI agents via MCP protocol
- **EZAuth flow** simplifies Cognito setup for OAuth authentication
- **Multiple targets** can be registered, each exposing different Lambda functions
- **Tool schemas** define how Lambda functions are exposed to agents
- **MCP client** must remain active while agents are using tools

## Next Steps

- Register your own Lambda functions with custom tool schemas
- Explore S3-based tool schemas for larger configurations
- Integrate with other AI agent frameworks that support MCP
- Set up monitoring and logging for production deployments

## Additional Resources

- [AWS Bedrock AgentCore Documentation](https://docs.aws.amazon.com/bedrock/)
- [Model Context Protocol Specification](https://modelcontextprotocol.io/)
- [Strands Agents Documentation](https://github.com/strands-ai/strands-agents)
- [Bedrock AgentCore Starter Toolkit](https://github.com/aws-samples/bedrock-agentcore-starter-toolkit)

---

**Note**: This is a sample implementation. For production use, ensure proper error handling, security best practices, and resource management.